<a href="https://colab.research.google.com/github/patricku93/ColabForFI/blob/main/ZIP_exports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import re
import os
import time
import html
import datetime
import zipfile
import pandas as pd
from google.colab import files
from ipywidgets import interact, widgets
from ipywidgets import interact, widgets, Layout, VBox, HBox
from IPython.display import display, HTML

# Ideen für Improvement:
# 1. Loop all over all exports and compare "Filename_xy" with "Filename_xy_WL100"
# Und zeige versionen missmatch wo version_xy < version_xy_WL100
# 2. Edit specfic values in export.json?
# 3. Show more/different Values?
# 4. Compare more than 2 files (needed?)
# 5. tbc

# Replace HTML entitiys in String with normal character
def html_to_regular(input_string):
    output_string = ""
    # Regular expression pattern to match HTML entities within words
    entity_pattern = r'\b&[a-zA-Z]+;\b'

    # Find all HTML entities within words in the input string
    matches = re.findall(entity_pattern, input_string)

    # Loop through the matches and decode them
    for match in matches:
        decoded_entity = html.unescape(match)
        output_string = input_string.replace(match, decoded_entity)
    if output_string:
        return output_string
    else:
        return input_string


report_list = []
def extract_json_from_zip(path):
    reports = []
    archive = zipfile.ZipFile(path, 'a')
    # load variable bericht with the export.json from the .zip file
    bericht = archive.open('export.json')
    data = json.load(bericht)

    for elem in data["elements"]:
        version_max = 0
        new_or_old = elem['type']
        report = {  "ID": "",
                    "NAME": "",
                    "VERSION": "",
                    "LAST_EDIT_AT": "",
                    "LAST_EDIT_BY": "",
                    "DELETE": "",
                    "EXPORTFILE_NAME": html_to_regular(archive.filename),
                    "EXPORTFILE_DATE" : datetime.datetime.fromtimestamp(os.path.getctime(path)).strftime("%d/%m/%Y, %H:%M:%S")
                    }
        if new_or_old in ["REPORT_TEMPLATE", "STATISCHE_BERICHTSSCHABLONE"]:
            report["ID"] = elem["elementId"]
            report["DELETE"] = data["deleteElements"]
            report["NAME"] = elem["elementName"]
            report["LAST_EDIT_BY"] = elem["freigaben"][-1]["user"]
            report["LAST_EDIT_AT"] = datetime.datetime.fromtimestamp(int(elem["freigaben"][0]["timestamp"])//1000).strftime("%d/%m/%Y, %H:%M:%S")

            if new_or_old == "STATISCHE_BERICHTSSCHABLONE":
                for version in elem['versions']:
                    if version['versionsnummer'] > version_max:
                        version_max = version['versionsnummer']
                    else:
                        continue
            else:
                for version in elem['versions']:
                    if version['version'] > version_max:
                        version_max = version['version']
                    else:
                        continue
            report["VERSION"] = version_max

            reports.append(report)

    return reports

def delete_file_copies(uploaded_file):
  for file in os.listdir(os.getcwd()):
    pattern = re.compile(r" \([0-9]+\)", re.IGNORECASE)
    file_name = re.sub(pattern, '', uploaded_file)
    if file_name == file:
      continue
    if file_name[:-4] in file:
      os.remove(file)
    else:
      continue
  return file_name

# Function to prompt the user for file upload while checking if it exists
# and delete copies with similiar name if needed
def prompt_for_upload():
    uploaded_files = files.upload()
    uploaded_files = delete_file_copies(list(uploaded_files.keys())[0])
    return uploaded_files if uploaded_files else None

# Prompt the user to upload the first ZIP file and get its name
zip1_name = prompt_for_upload()
while zip1_name is None:
    zip1_name = prompt_for_upload()

# Prompt the user to upload the second ZIP file and get its name
zip2_name = prompt_for_upload()
while zip2_name is None:
    zip2_name = prompt_for_upload()

# Extract JSON data from the uploaded ZIP files
json_data1 = extract_json_from_zip(zip1_name)
json_data2 = extract_json_from_zip(zip2_name)

# Create DataFrames for each JSON file
df1 = pd.DataFrame(json_data1)
df2 = pd.DataFrame(json_data2)

# Transpose the DataFrames to have reports as columns
df1 = df1.transpose()
df2 = df2.transpose()

# Rename columns to the value of the "NAME" key
df1.columns = df1.loc['NAME']
df2.columns = df2.loc['NAME']

# Drop the "NAME" row
df1.drop('NAME', inplace=True)
df2.drop('NAME', inplace=True)

# Create a widget to select the report or view all reports
report_selector = widgets.Dropdown(
    options=['All Reports'] + list(df1.columns.intersection(df2.columns)),  # Include an option for all reports
    description='Select Report:',
    disabled=False
)

# Function to display the selected reports
def display_reports(selected_report):
    if selected_report == 'All Reports':
        display(HTML(f'<h2>{zip1_name}</h2>'))
        display(df1)
        display(HTML(f'<h2>{zip2_name}</h2>'))
        display(df2)
    elif selected_report in df1.columns and selected_report in df2.columns:
        combined_df = pd.concat([df1[selected_report], df2[selected_report]], axis=1)
        display(combined_df)

# Function to export displayed results to an Excel file
def export_to_excel(button):
    selected_report = report_selector.value
    if selected_report == 'All Reports':
        with pd.ExcelWriter('comparison_results.xlsx') as writer:
            df1.to_excel(writer, sheet_name=f'{zip1_name}_Reports')
            df2.to_excel(writer, sheet_name=f'{zip2_name}_Reports')
    elif selected_report in df1.columns and selected_report in df2.columns:
        with pd.ExcelWriter('comparison_results.xlsx') as writer:
            combined_df = pd.concat([df1[selected_report], df2[selected_report]], axis=1)
            combined_df.to_excel(writer, sheet_name=f'{selected_report}_Comparison')
    print("Exported to Excel: comparison_results.xlsx")

# Create a button for exporting to Excel
export_button = widgets.Button(
    description='Export to Excel',
    button_style='success'
)
export_button.on_click(export_to_excel)

# Interact to choose and display reports or all reports
interact(display_reports, selected_report=report_selector)
display(export_button)



Saving Kontoauszug Del Pake.zip to Kontoauszug Del Pake.zip


Saving Kontoauszug Alle.zip to Kontoauszug Alle (1).zip


interactive(children=(Dropdown(description='Select Report:', options=('All Reports', '101 Kontoauszugsstatisti…

Button(button_style='success', description='Export to Excel', style=ButtonStyle())